In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer

years = ['2015', '2016', '2017', '2018', '2019']
data_frames = []
for year in years:
    df = pd.read_excel(f'TripDataset/Dataset/{year}.xlsx')  # Replace with actual file names
    df['Year'] = year  # Add year column for temporal tracking
    data_frames.append(df)

data = pd.concat(data_frames, ignore_index=True)
data = data.sort_values(by='Year')


In [2]:
# Define columns
numerical_cols = ['Age', 'Number of nights in CITY', 'Number of visits to CITY', 'Tour price', 'Airfare', 
                  'Total expenditures', 'Accommodation expenses', 'Food and drink expenses', 
                  'Transportation expenses in CITY', 'Entertainment expenses', 'Shopping expenses', 
                  'Other expenditures', 'Purpose of visit to CITY']
numerical_cols_without_target = ['Age', 'Number of visits to CITY', 'Tour price', 'Airfare', 
                  'Total expenditures', 'Accommodation expenses', 'Food and drink expenses', 
                  'Transportation expenses in CITY', 'Entertainment expenses', 'Shopping expenses', 
                  'Other expenditures']
target_cols = ['Purpose of visit to CITY', 'Number of nights in CITY']
true_categorical_cols = ['Nationality', 'Country of residence', 'Gender', 'Immigration airport', 
                        'Travel type', 'Having mileage or not', 'Intention to revisit CITY', 
                        'Most desired place', 'Most satisfied place']
potentially_binary_cols = ['Companion_1', 'Companion_2', 'Companion_3', 'Companion_4', 'Companion_5', 'Companion_6', 
                          'Accommodation facilities 1', 'Accommodation facilities 2', 'Accommodation facilities 3', 
                          'Accommodation facilities 4', 'Accommodation facilities 5', 'Accommodation facilities 6', 
                          'Accommodation facilities 7', 'Accommodation facilities 8', 
                          'Transportation 01', 'Transportation 02', 'Transportation 03', 'Transportation 04', 
                          'Transportation 05', 'Transportation 06', 'Transportation 07', 'Transportation 08', 
                          'Transportation 09', 'Transportation 10', 'Transportation 11', 'Transportation 12', 
                          'Transportation 13', 'Transportation 14', 'Transportation 15', 'Transportation 16', 
                          'Transportation 17', 'Activity 01', 'Activity 02', 'Activity 03', 'Activity 04', 'Activity 05', 
                          'Activity 06', 'Activity 07', 'Activity 08', 'Activity 09', 'Activity 10', 'Activity 11', 
                          'Activity 12', 'Activity 13', 'Activity 14', 'Activity 15', 'Activity 16', 'Activity 17', 
                          'Activity 18', 'Activity 19', 'Activity 20', 'Activity 21', 'Activity 22', 'Destination 01', 
                          'Destination 02', 'Destination 03', 'Destination 04', 'Destination 05', 'Destination 06', 
                          'Destination 07', 'Destination 08', 'Destination 09', 'Destination 10', 'Destination 11', 
                          'Destination 12', 'Destination 13', 'Destination 14', 'Destination 15', 'Destination 16', 
                          'Destination 17', 'Destination 18', 'Destination 19', 'Destination 20', 'Destination 21', 
                          'Destination 22', 'Destination 23', 'Destination 24', 'Activity_Destination 01', 
                          'Activity_Destination 02', 'Activity_Destination 03', 'Activity_Destination 04', 
                          'Activity_Destination 05', 'Activity_Destination 06', 'Activity_Destination 07', 
                          'Activity_Destination 08', 'Activity_Destination 09', 'Activity_Destination 10', 
                          'Activity_Destination 11', 'Activity_Destination 12', 'Activity_Destination 13', 
                          'Activity_Destination 14', 'Activity_Destination 15', 'Activity_Destination 16', 
                          'Activity_Destination 17', 'Activity_Destination 18', 'Activity_Destination 19', 
                          'Activity_Destination 20', 'Activity_Destination 21', 'Activity_Destination 22', 
                          'Activity_Destination 23', 'Activity_Destination 24', 'Trigger 01', 'Trigger 02', 
                          'Trigger 03', 'Trigger 04', 'Trigger 05', 'Trigger 06', 'Trigger 07', 'Trigger 08', 
                          'Trigger 09', 'Trigger 10', 'Trigger 11', 'Trigger 12', 'Trigger 13', 'Trigger 14', 
                          'Trigger 15', 'Trigger 16', 'Trigger 17', 'Trigger 18', 'Trigger 19', 'Trigger 20', 
                          'Trigger 21', 'Trigger 22', 'Trigger 23', 'Trigger 24', 'Trigger 25', 'Trigger 26', 
                          'Trigger 27', 'Trigger 28', 'Trigger 29', 'Attractions 01', 'Attractions 02', 
                          'Attractions 03', 'Attractions 04', 'Attractions 05', 'Attractions 06', 'Attractions 07', 
                          'Attractions 08', 'Attractions 09', 'Attractions 10', 'Attractions 11', 'Attractions 12', 
                          'Attractions 13', 'Attractions 14', 'Attractions 15', 'Attractions 16', 'Attractions 17', 
                          'Attractions 18', 'Attractions 19', 'Attractions 20', 'Attractions 21', 'Attractions 22', 
                          'Attractions 23', 'Attractions 24', 'Attractions 25', 'Attractions 26', 'Attractions 27']
ordinal_cols = ['Satisfaction level', 'Intention to revisit CITY', 'Satisfaction level by item 01', 
                'Satisfaction level by item 02', 'Satisfaction level by item 03', 'Satisfaction level by item 04', 
                'Satisfaction level by item 05', 'Satisfaction level by item 06', 'Satisfaction level by item 07', 
                'Satisfaction level by item 08', 'Satisfaction level by item 09', 'Satisfaction level by item 10', 
                'Satisfaction level by item 11']


In [3]:
# Imputing missing values
num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='most_frequent')
data[numerical_cols_without_target] = num_imputer.fit_transform(data[numerical_cols_without_target])
data[true_categorical_cols] = cat_imputer.fit_transform(data[true_categorical_cols])
data[potentially_binary_cols] = cat_imputer.fit_transform(data[potentially_binary_cols])

data[true_categorical_cols] = data[true_categorical_cols].astype(str)


In [7]:
# 初始化 OneHotEncoder
ohe = OneHotEncoder(handle_unknown='ignore')

# 对 true_categorical_cols 进行独热编码
cat_encoded = pd.DataFrame(
    ohe.fit_transform(data[true_categorical_cols]).toarray(),  # 转为密集矩阵
    columns=ohe.get_feature_names_out(true_categorical_cols)
)

ord_encoder = OrdinalEncoder()
ord_encoded = pd.DataFrame(ord_encoder.fit_transform(data[ordinal_cols]), 
                           columns=ordinal_cols)
binary_encoded = data[potentially_binary_cols]
scaler = StandardScaler()
num_scaled = pd.DataFrame(scaler.fit_transform(data[numerical_cols_without_target]), 
                          columns=numerical_cols_without_target)
data_processed = pd.concat([num_scaled, cat_encoded, binary_encoded, ord_encoded, data['Number of nights in CITY'], data['Purpose of visit to CITY']], axis=1)

In [10]:
data['Number of nights in CITY'] = data['Number of nights in CITY'].replace(0, 1e-1)
data_processed['Expenditure_per_night'] = data['Total expenditures'] / data['Number of nights in CITY']

data_processed = data_processed.fillna(data_processed.mean())  # 填充数值型特征的缺失值

data_processed.to_pickle("data/data_processed.pkl")